In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy.stats import multivariate_normal

In [4]:
# multivariate normal documentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html


def E_step(data,pi,mu,sigma):
    """
    
    Source: https://towardsdatascience.com/latent-variables-expectation-maximization-algorithm-fb15c4e0f32c
    """
    N = data.shape[0] # number of data-points
    K = pi.shape[0] # number of clusters
    d = mu.  shape[1] # dimension/ attributes of each data point
    gamma = np.zeros((N, K)) # posterior
    
    for nk in range(K):
        gamma[:, nk] = pi[nk]*multivariate_normal.pdf(data, mean=mu[nk], cov=sigma[nk])
        
    return gamma/np.sum(gamma, axis=1, keepdims=True) # calculate posterior for each cluster
    

In [5]:
def M_step(data, gamma):
    
    """
    
    Source: https://towardsdatascience.com/latent-variables-expectation-maximization-algorithm-fb15c4e0f32c
    """
    
    N, D = data.shape
    K = gamma.shape[1] # use the posterior shape calculated in E-step to determine the number of clusters
    pi = np.zeros(K)
    mu = np.zeros({K,D})
    sigma = np.zeros({K,D,D})
    
    for ik in range(K): 
        n_k = gamma[:, ik].sum() # use the definition of n_k
        pi[ik] = n_k/N # definition of the weights
        elements = np.reshape(gamma[:, ik], (gamma.shpae[0], 1))
        
        # get each columns and reshape it (K, 1) form so hat later broadcasting is possible.
        mu[ik,:] = (np.multiply( elements, data)).sum(axis=0)/ n_k
        sigma_sum = 0
        for i in range(N):
            var = data[i] - mu[ik]
            sigma_sum = sigma_sum + gamma[i, ik]*np.outer(var, var) # outer product creatures the covariance matrix
        simga[ik, :] = sigma_sum/n_k
    return pi, mu, sigma

        
    

In [6]:
def elbo(data, gamma, pi, mu, sigma):
    
    N = data.shape[0] # number of data points
    K = gamma.shape[1] # number of clusters
    d = data.shape[1] # dimension of each object
    
    loss = 0
    for i in range(N):
        x = data[i]
        for k in range(K):
            post_dist = gamma[i, k] # p(z_i=k|x) = gamma_ik
            log_lik = np.log(multivariate_normal.pdf(x, mean=mu[k, :], cov=sigma[k, :, :]) +le-20) # log p(x|z)
            log_q = np.log(gamma[i,k] + le-20) # log q(z) = log p(z_i=k|x)
            log_pz = np.log(pi[k] + le-20) # log p(z_k =1) =\pi_k
            loss = (loss + np.multiply(pos_dis, log_pz) + np.multiply(pos_dist, log_lik) +
                   np.multiply(post_dist, -log_q))
        
    print("check loss: "), loss

    return loss


In [27]:
def train_loop(data, K, tolerance=1e-3, max_iter=50, restart=10):
    
    if len(data.shape)==1:
        N = data.shape
        d = 0
    else:
        N, d = data.shape
    
    
    elbo_best = -np.inf # loss set to the lowest value 
    pi_best = None
    mu_best = None
    sigma_best = None
    gamma_f = None
    for _ in range(restart):
        pi = np.ones(K) / K # if 3 clusters then an array of [.33, .33, .33] 
                                # the sum of pi's should be one
                            # that's why normalized  
                
        mu = np.random.rand(K, d) # no condition on 
        sigma = np.tile(np.eye(d), (K, 1, 1)) # to initialize sigma we first start with ones only at the diagonals
                                              # the sigmas (Covariance matrices) are postive semi-definite and symmetric  
        last_iter_loss = None
        all_losses = []
        
        try:
            for i in range(max_iter):
                gamma = E_step(data, pi, mu, sigma)
                pi, mu, sigma = M_step(data, gamma)
                loss = elbo(data, gamma, pi, mu, sigma)
                
                if loss > elbo_best:
                    elbo_best = loss
                    pi_best = pi 
                    mu_best = mu
                    sigma_best = sigma
                    gamma_f = gamma
                
                if last_iter_loss and abs((loss-last_iter_loss)/last_iter_loss) < tolerance: # insignificant improvement
                    break 
                    
            last_iter_loss = loss
            all_losses.append(loss)
            
        except np.linalg.LinAlgError: # avoid the delta function situation 
            pass 
        
    return elbo_best, pi_best, mu_best, sigma_best, all_losses, gamma_f    

In [28]:
# generate a sample
X1 = np.random.normal(loc=20, scale=5, size=3000)
X2 = np.random.normal(loc=40, scale=5, size=7000)
X = np.hstack((X1, X2))

In [29]:
train_loop(data=X, K=3)

error: (il>=1&&il<=n) failed for 6th keyword il: dsyevr:il=1

In [25]:
len(X.shape)

1

In [26]:
this_test = (5,1)

In [20]:
k, d = this_test

ValueError: not enough values to unpack (expected 2, got 1)